In [153]:
import tensorflow as tf
import numpy as np
import pandas as pd
from __future__ import print_function
import os
from io import BytesIO
import numpy as np
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML

# 1. Create your first Graph

In [72]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

In [73]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
sess.run(f)
sess.close()

In [74]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [75]:
print result

42


In [76]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print f.eval()

42


# 2. Managing Graphs

In [77]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [78]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

# 3. Lifecycle of a Node Value

In [79]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# x and w will be evaluated twice
with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

10
15


In [80]:
with tf.Session() as sess:
    [y_val, z_val] = sess.run([y, z])
    print y_val
    print z_val

10
15


# 4. Linear Regression with Tensorflow

In [81]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [82]:
housing = fetch_california_housing()

In [83]:
(m, n) = housing.data.shape

In [84]:
housing_data_with_bias = np.c_[np.ones((m, 1)), housing.data]

In [85]:
X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [86]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [87]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

# 5. Linear Regression with Gradient Descent

In [88]:
from sklearn.preprocessing import StandardScaler
(m, n) = housing.data.shape
transformed_housing_data = StandardScaler().fit_transform(housing.data)
housing_data_with_bias = np.c_[np.ones((m, 1)), transformed_housing_data]

In [91]:
n_epochs = 1000
learning_rate = 0.01
graph = tf.Graph()
with graph.as_default():
    X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name="theta")
    y_predictions = tf.matmul(X, theta, name="predictions")
    error = y_predictions - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    gradients = (2 / m) * tf.matmul(tf.transpose(X), error, name="gradients") # must be 2 / m, not 2.0/m
    training_op = tf.assign(theta, theta - learning_rate * gradients)
    init = tf.global_variables_initializer()

with tf.Session(graph=graph) as sess:
    sess.run(init)
    for i in range(n_epochs):
        if i % 100 == 0:
            mse_val = sess.run(mse)
            print("epoch: {}, mse: {}".format(i, mse_val))
        sess.run(training_op)
    best_theta = theta.eval()
print best_theta

epoch: 0, mse: 8.92024421692
epoch: 100, mse: 8.92024421692
epoch: 200, mse: 8.92024421692
epoch: 300, mse: 8.92024421692
epoch: 400, mse: 8.92024421692
epoch: 500, mse: 8.92024421692
epoch: 600, mse: 8.92024421692
epoch: 700, mse: 8.92024421692
epoch: 800, mse: 8.92024421692
epoch: 900, mse: 8.92024421692
[[ 0.63291645]
 [-0.438591  ]
 [-0.07727122]
 [-0.48363018]
 [-0.9213624 ]
 [ 0.28029251]
 [-0.47783971]
 [-0.82286477]
 [ 0.70768929]]


In [92]:
# With tensorflow GradientDescentOptimizer
n_epochs = 1000
learning_rate = 0.01
graph = tf.Graph()
with graph.as_default():
    X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X") # data has to been standard scaled
    y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name="theta")
    y_predictions = tf.matmul(X, theta, name="predictions")
    error = y_predictions - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    grad_loss = tf.Variable(tf.zeros(n + 1), name="grad_loss")
    gradients = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = gradients.minimize(mse)
    init = tf.global_variables_initializer()

with tf.Session(graph=graph) as sess:
    sess.run(init)
    for i in range(n_epochs):
        if i % 100 == 0:
            mse_val = sess.run(mse)
            print("epoch: {}, mse: {}".format(i, mse_val))
        sess.run(training_op)
    best_theta = sess.run(theta)
print best_theta

epoch: 0, mse: 7.51960229874
epoch: 100, mse: 0.829821288586
epoch: 200, mse: 0.676665604115
epoch: 300, mse: 0.632684767246
epoch: 400, mse: 0.602546274662
epoch: 500, mse: 0.580851435661
epoch: 600, mse: 0.565189063549
epoch: 700, mse: 0.553876578808
epoch: 800, mse: 0.545703113079
epoch: 900, mse: 0.539797604084
[[ 2.06855226]
 [ 0.78839767]
 [ 0.15214697]
 [-0.11072706]
 [ 0.14386316]
 [ 0.00818035]
 [-0.04127223]
 [-0.6951474 ]
 [-0.65728396]]


# 6. Feed data to graph

In [95]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

In [100]:
with tf.Session() as sess:
    B_val1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val2 = B.eval(feed_dict={A: [[1, 2, 3], [4, 5, 6]]})

In [140]:
# Linear Regression with placeholder
n_epochs = 1000
learning_rate = 0.01
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name="theta")
    y_predictions = tf.matmul(X, theta, name="predictions")
    error = y_predictions - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    grad_loss = tf.Variable(tf.zeros(n + 1), name="grad_loss")
    gradients = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = gradients.minimize(mse)
    init = tf.global_variables_initializer()

x_batch = housing_data_with_bias
y_batch = housing.target.reshape(-1, 1)
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for i in range(n_epochs):
        if i % 100 == 0:
            mse_val = sess.run(mse, feed_dict={X: x_batch, y: y_batch})
            print("epoch: {}, mse: {}".format(i, mse_val))
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch})
    best_theta = sess.run(theta)
print best_theta

epoch: 0, mse: 5.91423034668
epoch: 100, mse: 0.683947265148
epoch: 200, mse: 0.58988904953
epoch: 300, mse: 0.57032763958
epoch: 400, mse: 0.557835936546
epoch: 500, mse: 0.548844754696
epoch: 600, mse: 0.54230594635
epoch: 700, mse: 0.537540435791
epoch: 800, mse: 0.534061789513
epoch: 900, mse: 0.531518816948
[[ 2.06855249]
 [ 0.827878  ]
 [ 0.14413993]
 [-0.21414192]
 [ 0.24205476]
 [ 0.00475607]
 [-0.04143694]
 [-0.71645463]
 [-0.6845808 ]]


# 7. Saving the model

In [143]:
n_epochs = 1000
learning_rate = 0.01
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name="theta")
    y_predictions = tf.matmul(X, theta, name="predictions")
    error = y_predictions - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    grad_loss = tf.Variable(tf.zeros(n + 1), name="grad_loss")
    gradients = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = gradients.minimize(mse)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

x_batch = housing_data_with_bias
y_batch = housing.target.reshape(-1, 1)
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for i in range(n_epochs):
        if i % 100 == 0:
            mse_val = sess.run(mse, feed_dict={X: x_batch, y: y_batch})
            print("epoch: {}, mse: {}".format(i, mse_val))
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch})
    best_theta = sess.run(theta)
    save_path = saver.save(sess, "/tmp/tensorflow/linear_model.ckpt")
print best_theta
print save_path

epoch: 0, mse: 13.3487520218
epoch: 100, mse: 1.02892303467
epoch: 200, mse: 0.694108486176
epoch: 300, mse: 0.640715181828
epoch: 400, mse: 0.609049379826
epoch: 500, mse: 0.586402952671
epoch: 600, mse: 0.56992828846
epoch: 700, mse: 0.557907581329
epoch: 800, mse: 0.549122691154
epoch: 900, mse: 0.542689681053
[[ 2.06855249]
 [ 0.83025265]
 [ 0.15951233]
 [-0.19070593]
 [ 0.2103941 ]
 [ 0.01031574]
 [-0.04278753]
 [-0.602144  ]
 [-0.56919539]]
/tmp/tensorflow/linear_model.ckpt


In [146]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, save_path)
    best_theta = sess.run(theta)
    print best_theta

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/linear_model.ckpt
[[ 2.06855249]
 [ 0.83025265]
 [ 0.15951233]
 [-0.19070593]
 [ 0.2103941 ]
 [ 0.01031574]
 [-0.04278753]
 [-0.602144  ]
 [-0.56919539]]


# 8. Tensorboard

In [148]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "/tmp/tensorflow/{}/run-{}/".format(root_logdir, now)

In [163]:
n_epochs = 1000
learning_rate = 0.01
graph = tf.Graph()
with graph.as_default():
    X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name="theta")
    y_predictions = tf.matmul(X, theta, name="predictions")
    with tf.name_scope("grad_loss") as scope:
        error = y_predictions - y
        mse = tf.reduce_mean(tf.square(error), name="mse")
        grad_loss = tf.Variable(tf.zeros(n + 1), name="grad_loss")
        gradients = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        training_op = gradients.minimize(mse)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    # add summary
    mse_summary = tf.summary.scalar("MSE", mse)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

x_batch = housing_data_with_bias
y_batch = housing.target.reshape(-1, 1)
with tf.Session(graph=graph) as sess:
    sess.run(init)
    for i in range(n_epochs):
        if i % 100 == 0:
            (mse_val, summary_str) = sess.run([mse, mse_summary], feed_dict={X: x_batch, y: y_batch})
            print("epoch: {}, mse: {}".format(i, mse_val))
            file_writer.add_summary(summary_str, i)
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch})
    best_theta = sess.run(theta)
    save_path = saver.save(sess, "/tmp/tensorflow/linear_model.ckpt")
    print(save_path)
    file_writer.close()
    print(best_theta)

epoch: 0, mse: 12.4321737289
epoch: 100, mse: 0.681682825089
epoch: 200, mse: 0.542516052723
epoch: 300, mse: 0.536007881165
epoch: 400, mse: 0.532868504524
epoch: 500, mse: 0.530609071255
epoch: 600, mse: 0.528959214687
epoch: 700, mse: 0.527751922607
epoch: 800, mse: 0.526866614819
epoch: 900, mse: 0.526217103004
/tmp/tensorflow/linear_model.ckpt
[[  2.06855226e+00]
 [  8.32204282e-01]
 [  1.31969199e-01]
 [ -2.46595874e-01]
 [  2.79568762e-01]
 [  2.71917350e-04]
 [ -4.04995233e-02]
 [ -8.01107824e-01]
 [ -7.70911992e-01]]


# 9. Variable Scope

In [165]:
def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="max")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")